In [14]:
import pandas as pd
import plotly.express as px

# Define columns and dtypes
usecols = [
    'SOURCE_DESC', 'SECTOR_DESC', 'GROUP_DESC', 'COMMODITY_DESC',
    'YEAR', 'REFERENCE_PERIOD_DESC', 'VALUE', 'UNIT_DESC', 'STATE_ALPHA',
    'STATISTICCAT_DESC'
]
dtypes = {
    'SOURCE_DESC': 'category',
    'SECTOR_DESC': 'category',
    'GROUP_DESC': 'category',
    'COMMODITY_DESC': 'category',
    'YEAR': 'int32',
    'REFERENCE_PERIOD_DESC': 'category',
    'UNIT_DESC': 'category',
    'STATE_ALPHA': 'category',
    'STATISTICCAT_DESC': 'category'
}

# Load and filter data in chunks
chunksize = 100000
df_list = []

for chunk in pd.read_csv(
    '/data/qs.crops_20250404.txt',
    sep='\t',
    encoding='latin1',
    usecols=usecols,
    dtype=dtypes,
    chunksize=chunksize
):
    chunk['VALUE'] = pd.to_numeric(chunk['VALUE'].str.replace(',', ''), errors='coerce')
    filtered_chunk = chunk[
        (chunk['COMMODITY_DESC'] == 'CORN') &
        (chunk['YEAR'].between(2020, 2024)) &
        (chunk['SECTOR_DESC'] == 'CROPS') &
        (chunk['STATISTICCAT_DESC'] == 'YIELD')  # Use 'PRICE RECEIVED' for prices
    ]
    df_list.append(filtered_chunk)

# Concatenate and ensure VALUE is numeric
df = pd.concat(df_list, ignore_index=True)
df['VALUE'] = pd.to_numeric(df['VALUE'], errors='coerce')

# Aggregate by state and year (mean VALUE)
df_agg = df.groupby(['STATE_ALPHA', 'YEAR'])['VALUE'].mean().reset_index()

# Save aggregated data
df_agg.to_csv('data/corn_yield_by_state_2020_2024.csv', index=False)

# Create interactive choropleth map
fig = px.choropleth(
    df_agg,
    locations='STATE_ALPHA',
    locationmode='USA-states',
    color='VALUE',
    hover_name='STATE_ALPHA',
    animation_frame='YEAR',
    color_continuous_scale='YlGn',
    scope='usa',
    title='Corn Yield by State (2020-2024)',
    labels={'VALUE': 'Yield (BU / ACRE)'},
)

# Update layout
fig.update_layout(
    geo=dict(framecolor='black', showframe=True),
    width=800,
    height=600
)

# Save interactive map
fig.write_html('images/corn_yield_map.html')

# Static image for 2023
df_2023 = df_agg[df_agg['YEAR'] == 2023]
fig_2023 = px.choropleth(
    df_2023,
    locations='STATE_ALPHA',
    locationmode='USA-states',
    color='VALUE',
    hover_name='STATE_ALPHA',
    color_continuous_scale='YlGn',
    scope='usa',
    title='Corn Yield by State (2023)',
    labels={'VALUE': 'Yield (BU / ACRE)'},
)
fig_2023.update_layout(
    geo=dict(framecolor='black', showframe=True),
    width=800,
    height=600
)
fig_2023.write_image('images/corn_yield_map_2023.png', scale=2)

FileNotFoundError: [Errno 2] No such file or directory: '../data/qs.crops_20250404.txt'

In [16]:
import pandas as pd
import plotly.express as px

# Define columns and dtypes
usecols = [
    'SOURCE_DESC', 'SECTOR_DESC', 'GROUP_DESC', 'COMMODITY_DESC',
    'YEAR', 'REFERENCE_PERIOD_DESC', 'VALUE', 'UNIT_DESC', 'STATE_ALPHA',
    'STATISTICCAT_DESC'
]
dtypes = {
    'SOURCE_DESC': 'category',
    'SECTOR_DESC': 'category',
    'GROUP_DESC': 'category',
    'COMMODITY_DESC': 'category',
    'YEAR': 'int32',
    'REFERENCE_PERIOD_DESC': 'category',
    'UNIT_DESC': 'category',
    'STATE_ALPHA': 'category',
    'STATISTICCAT_DESC': 'category'
}

# Load and filter data in chunks
chunksize = 100000
df_list = []

crops = ['CORN', 'SOYBEANS', 'WHEAT', 'COTTON']  # Add more crops as needed
for chunk in pd.read_csv(
    'data/qs.crops_20250404.txt',
    sep='\t',
    encoding='latin1',
    usecols=usecols,
    dtype=dtypes,
    chunksize=chunksize
):
    chunk['VALUE'] = pd.to_numeric(chunk['VALUE'].str.replace(',', ''), errors='coerce')
    filtered_chunk = chunk[
        (chunk['COMMODITY_DESC'].isin(crops)) &
        (chunk['YEAR'].between(2020, 2024)) &
        (chunk['SECTOR_DESC'] == 'CROPS') &
        (chunk['STATISTICCAT_DESC'] == 'YIELD')  # Use 'PRICE RECEIVED' for prices
    ]
    df_list.append(filtered_chunk)

# Concatenate and ensure VALUE is numeric
df = pd.concat(df_list, ignore_index=True)
df['VALUE'] = pd.to_numeric(df['VALUE'], errors='coerce')

# Aggregate by state, year, and commodity
df_agg = df.groupby(['STATE_ALPHA', 'YEAR', 'COMMODITY_DESC'])['VALUE'].mean().reset_index()

# Save aggregated data
df_agg.to_csv('data/crop_yield_by_state_2020_2024.csv', index=False)

# Create base choropleth map
fig = px.choropleth(
    df_agg,
    locations='STATE_ALPHA',
    locationmode='USA-states',
    color='VALUE',
    hover_name='STATE_ALPHA',
    animation_frame='YEAR',
    color_continuous_scale='YlGn',
    scope='usa',
    title='Crop Yield by State (2020-2024)',
    labels={'VALUE': 'Yield (Units Vary by Crop)'},
)

# Add radio buttons for crop selection
buttons = []
for crop in crops:
    buttons.append(
        dict(
            label=crop,
            method='update',
            args=[{'visible': [c == crop for c in df_agg['COMMODITY_DESC']]},
                  {'title': f'{crop} Yield by State (2020-2024)'}]
        )
    )

fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.1,
            xanchor='left',
            y=1.1,
            yanchor='top'
        )
    ],
    geo=dict(framecolor='black', showframe=True),
    width=800,
    height=600
)

# Save interactive map
fig.write_html('images/crop_yield_map.html')

# Static image for 2023 (Corn as example)
df_2023_corn = df_agg[(df_agg['YEAR'] == 2023) & (df_agg['COMMODITY_DESC'] == 'CORN')]
fig_2023 = px.choropleth(
    df_2023_corn,
    locations='STATE_ALPHA',
    locationmode='USA-states',
    color='VALUE',
    hover_name='STATE_ALPHA',
    color_continuous_scale='YlGn',
    scope='usa',
    title='Corn Yield by State (2023)',
    labels={'VALUE': 'Yield (BU / ACRE)'},
)
fig_2023.update_layout(
    geo=dict(framecolor='black', showframe=True),
    width=800,
    height=600
)
fig_2023.write_image('images/corn_yield_map_2023.png', scale=2)

In [18]:
import pandas as pd
import plotly.express as px

# 1. Data Loading and Processing
def load_and_filter_data(file_path, crops, years=(2020, 2024)):
    usecols = [
        'SOURCE_DESC', 'SECTOR_DESC', 'GROUP_DESC', 'COMMODITY_DESC',
        'YEAR', 'REFERENCE_PERIOD_DESC', 'VALUE', 'UNIT_DESC', 'STATE_ALPHA',
        'STATISTICCAT_DESC'
    ]
    dtypes = {
        'SOURCE_DESC': 'category', 'SECTOR_DESC': 'category', 'GROUP_DESC': 'category',
        'COMMODITY_DESC': 'category', 'YEAR': 'int32', 'REFERENCE_PERIOD_DESC': 'category',
        'UNIT_DESC': 'category', 'STATE_ALPHA': 'category', 'STATISTICCAT_DESC': 'category'
    }
    chunksize = 100000
    df_list = []

    for chunk in pd.read_csv(
        file_path, sep='\t', encoding='latin1', usecols=usecols, dtype=dtypes, chunksize=chunksize
    ):
        chunk['VALUE'] = pd.to_numeric(chunk['VALUE'].str.replace(',', ''), errors='coerce')
        filtered_chunk = chunk[
            (chunk['COMMODITY_DESC'].isin(crops)) &
            (chunk['YEAR'].between(years[0], years[1])) &
            (chunk['SECTOR_DESC'] == 'CROPS') &
            (chunk['STATISTICCAT_DESC'] == 'YIELD')  # Switch to 'PRICE RECEIVED' for prices
        ]
        df_list.append(filtered_chunk)

    df = pd.concat(df_list, ignore_index=True)
    df['VALUE'] = pd.to_numeric(df['VALUE'], errors='coerce')
    return df

# 2. Data Aggregation
def aggregate_data(df):
    df_agg = df.groupby(['STATE_ALPHA', 'YEAR', 'COMMODITY_DESC'])['VALUE'].mean().reset_index()
    return df_agg

# 3. Create Interactive Map
def create_interactive_map(df_agg, crops):
    fig = px.choropleth(
        df_agg,
        locations='STATE_ALPHA',
        locationmode='USA-states',
        color='VALUE',
        hover_name='STATE_ALPHA',
        animation_frame='YEAR',
        color_continuous_scale='YlGn',
        scope='usa',
        title='Crop Yield by State (2020-2024)',
        labels={'VALUE': 'Yield (Units Vary by Crop)'}
    )

    # Add radio buttons for crops
    buttons = []
    for i, crop in enumerate(crops):
        buttons.append(
            dict(
                label=crop,
                method='restyle',
                args=[{
                    'visible': [True if c == crop else False for c in df_agg['COMMODITY_DESC']],
                    'title': f'{crop} Yield by State (2020-2024)'
                }],
                args2=[{'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}]
            )
        )

    fig.update_layout(
        updatemenus=[dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.1,
            xanchor='left',
            y=1.1,
            yanchor='top'
        )],
        geo=dict(framecolor='black', showframe=True),
        width=800,
        height=600
    )
    return fig

# 4. Save Outputs
def save_outputs(fig, df_agg):
    fig.write_html('images/crop_yield_map.html')
    df_agg.to_csv('data/crop_yield_by_state_2020_2024.csv', index=False)
    # Static image for 2023 (Corn)
    df_2023_corn = df_agg[(df_agg['YEAR'] == 2023) & (df_agg['COMMODITY_DESC'] == 'CORN')]
    fig_2023 = px.choropleth(
        df_2023_corn,
        locations='STATE_ALPHA',
        locationmode='USA-states',
        color='VALUE',
        hover_name='STATE_ALPHA',
        color_continuous_scale='YlGn',
        scope='usa',
        title='Corn Yield by State (2023)',
        labels={'VALUE': 'Yield (BU / ACRE)'}
    )
    fig_2023.update_layout(geo=dict(framecolor='black', showframe=True), width=800, height=600)
    fig_2023.write_image('images/corn_yield_map_2023.png', scale=2)

# Main execution
if __name__ == "__main__":
    crops = ['CORN', 'SOYBEANS', 'WHEAT', 'COTTON']
    file_path = 'data/qs.crops_20250404.txt'
    
    # df = load_and_filter_data(file_path, crops)
    df_agg = aggregate_data(df)
    fig = create_interactive_map(df_agg, crops)
    save_outputs(fig, df_agg)

In [10]:
# df['STATISTICCAT_DESC'].value_counts()
df['STATISTICCAT_DESC'].unique()

array(['YIELD', 'STOCKS', 'SUCROSE', 'PRODUCTION', 'AREA HARVESTED',
       'AREA PLANTED', 'CONDITION, PREVIOUS YEAR', 'SALES',
       'PRICE RECEIVED', 'CONDITION, 5 YEAR AVG', 'AREA IN PRODUCTION',
       'CONDITION', 'PROGRESS', 'AREA PLANTED, NET', 'AREA BEARING',
       'AREA BEARING & NON-BEARING', 'AREA GROWN', 'PROGRESS, 5 YEAR AVG',
       'CAPACITY', 'MOISTURE', 'WATER APPLIED', 'AREA NON-BEARING',
       'MOISTURE, PREVIOUS YEAR', 'SALES IN ORGANIC MARKETS',
       'ACTIVE GINS', 'GINNED BALES',
       'INDEX FOR PRICE RECEIVED, 1910 - 1914', 'USAGE',
       'PROGRESS, PREVIOUS YEAR', 'SHRINK', 'ACTIVITY, PREVIOUS YEAR',
       'DAYS SUITABLE, PREVIOUS YEAR', 'INDEX FOR PRICE RECEIVED, 2011',
       'AREA NOT HARVESTED', 'POD COUNT', 'TAPS',
       'PRICE RECEIVED, PARITY', 'DAYS SUITABLE', 'INVENTORY',
       'HEIGHT, AVG, PREVIOUS YEAR', 'DAMAGE, PREVIOUS YEAR',
       'OPERATIONS', 'FARM USE', 'PRICE REACTION',
       'PRICE RECEIVED AFTER REPORT', 'AREA', 'DAMAGE, 5 YEA

In [5]:
import pandas as pd
df = pd.read_csv('data/qs.crops_20250404.txt', encoding='Latin1',sep='\t')

C:\Users\sujit\AppData\Local\Temp\ipykernel_7516\2717005088.py:2: DtypeWarning: Columns (19,20,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/qs.crops_20250404.txt', encoding='Latin1',sep='\t')


In [6]:
df.head(10)

,SOURCE_DESC,SECTOR_DESC,GROUP_DESC,COMMODITY_DESC,CLASS_DESC,PRODN_PRACTICE_DESC,UTIL_PRACTICE_DESC,STATISTICCAT_DESC,UNIT_DESC,SHORT_DESC,...,LOCATION_DESC,YEAR,FREQ_DESC,BEGIN_CODE,END_CODE,REFERENCE_PERIOD_DESC,WEEK_ENDING,LOAD_TIME,VALUE,CV_%
0,SURVEY,CROPS,FIELD CROPS,SOYBEANS,ALL CLASSES,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,YIELD,BU / ACRE,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",...,"MICHIGAN, SOUTHWEST, CASS",1972,ANNUAL,0,0,YEAR,NaN,2012-01-01 00:00:00,23.1,NaN
1,SURVEY,CROPS,FIELD CROPS,SOYBEANS,ALL CLASSES,ALL PRODUCTION PRACTICES,ON FARM,STOCKS,BU,"SOYBEANS, ON FARM - STOCKS, MEASURED IN BU",...,TENNESSEE,1965,POINT IN TIME,12,12,FIRST OF DEC,NaN,2012-01-01 00:00:00,"2,236,000",NaN
2,SURVEY,CROPS,FIELD CROPS,SUGARBEETS,ALL CLASSES,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,SUCROSE,PCT,"SUGARBEETS - SUCROSE, MEASURED IN PCT",...,"OHIO, NORTHWEST, PUTNAM",1983,ANNUAL,0,0,YEAR,NaN,2012-01-01 00:00:00,16.26,NaN
3,SURVEY,CROPS,FIELD CROPS,HAY,ALL CLASSES,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,PRODUCTION,TONS,"HAY - PRODUCTION, MEASURED IN TONS",...,"MISSOURI, NORTHWEST, ANDREW",1992,ANNUAL,0,0,YEAR,NaN,2012-01-01 00:00:00,"49,500",NaN
4,SURVEY,CROPS,FIELD CROPS,CORN,ALL CLASSES,ALL PRODUCTION PRACTICES,SILAGE,PRODUCTION,TONS,"CORN, SILAGE - PRODUCTION, MEASURED IN TONS",...,"NEW YORK, CENTRAL, CORTLAND",1991,ANNUAL,0,0,YEAR,NaN,2012-01-01 00:00:00,"184,200",NaN
5,SURVEY,CROPS,FIELD CROPS,HAY,ALFALFA,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,YIELD,TONS / ACRE,"HAY, ALFALFA - YIELD, MEASURED IN TONS / ACRE",...,"NEBRASKA, NORTHWEST, DEUEL",1965,ANNUAL,0,0,YEAR,NaN,2012-01-01 00:00:00,2.88,NaN
6,SURVEY,CROPS,FIELD CROPS,WHEAT,WINTER,NON-IRRIGATED,ALL UTILIZATION PRACTICES,PRODUCTION,BU,"WHEAT, WINTER, NON-IRRIGATED - PRODUCTION, MEA...",...,"SOUTH DAKOTA, CENTRAL, BUFFALO",2006,ANNUAL,0,0,YEAR,NaN,2012-01-01 00:00:00,"154,000",NaN
7,SURVEY,CROPS,FIELD CROPS,SUNFLOWER,ALL CLASSES,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,AREA HARVESTED,ACRES,SUNFLOWER - ACRES HARVESTED,...,"NORTH DAKOTA, CENTRAL, SHERIDAN",1984,ANNUAL,0,0,YEAR,NaN,2012-01-01 00:00:00,"69,000",NaN
8,SURVEY,CROPS,FIELD CROPS,HAY,ALFALFA,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,AREA HARVESTED,ACRES,"HAY, ALFALFA - ACRES HARVESTED",...,"OKLAHOMA, SOUTH CENTRAL, PONTOTOC",1993,ANNUAL,0,0,YEAR,NaN,2012-01-01 00:00:00,"2,000",NaN
9,SURVEY,CROPS,FIELD CROPS,WHEAT,ALL CLASSES,NON-IRRIGATED,ALL UTILIZATION PRACTICES,AREA HARVESTED,ACRES,"WHEAT, NON-IRRIGATED - ACRES HARVESTED",...,"WASHINGTON, SOUTHEAST",1975,ANNUAL,0,0,YEAR,NaN,2012-01-01 00:00:00,"951,100",NaN


In [7]:
df['VALUE'] = pd.to_numeric(df['VALUE'].str.replace(',', ''), errors='coerce')

In [11]:
# Filter for target crops and years
crops = ['CORN', 'SOYBEANS', 'WHEAT', 'COTTON']
df_filtered = df[
    (df['COMMODITY_DESC'].isin(crops)) &
    (df['YEAR'].between(2000, 2024)) &
    (df['SECTOR_DESC'] == 'CROPS') &
    (df['STATISTICCAT_DESC'] == 'YIELD')  # Use 'PRICE RECEIVED' for prices
]

# Separate survey and census data
df_survey = df_filtered[df_filtered['SOURCE_DESC'] == 'SURVEY']
df_census = df_filtered[df_filtered['SOURCE_DESC'] == 'CENSUS']

# Aggregate by state, year, and crop
df_survey_agg = df_survey.groupby(['STATE_ALPHA', 'YEAR', 'COMMODITY_DESC'])['VALUE'].mean().reset_index()
df_census_agg = df_census.groupby(['STATE_ALPHA', 'YEAR', 'COMMODITY_DESC'])['VALUE'].mean().reset_index()

In [18]:
import geopandas as gpd

# Load U.S. state boundaries from Natural Earth or similar
# Download from: https://www.naturalearthdata.com/downloads/110m-cultural-vectors/
states_gdf = gpd.read_file('data/ne_110m_admin_1_states_provinces.shp')  # Adjust path
states_gdf = states_gdf[states_gdf['iso_a2'] == 'US']  # Filter for U.S.
states_gdf = states_gdf[['postal', 'geometry']]  # Keep state code and geometry
states_gdf.rename(columns={'postal': 'STATE_ALPHA'}, inplace=True)

# Extract centroids for simplicity (lon, lat)
states_gdf['lon'] = states_gdf.geometry.centroid.x
states_gdf['lat'] = states_gdf.geometry.centroid.y

# Merge with aggregated data and convert to GeoDataFrame
df_survey_geo = df_survey_agg.merge(states_gdf[['STATE_ALPHA', 'lon', 'lat', 'geometry']], on='STATE_ALPHA', how='left')
df_census_geo = df_census_agg.merge(states_gdf[['STATE_ALPHA', 'lon', 'lat', 'geometry']], on='STATE_ALPHA', how='left')

# Convert to GeoDataFrame
gdf_survey = gpd.GeoDataFrame(df_survey_geo, geometry='geometry')
gdf_census = gpd.GeoDataFrame(df_census_geo, geometry='geometry')

# Save as GeoJSON for QGIS or browser use
gdf_survey.to_file('data/survey_crop_yield.geojson', driver='GeoJSON')
gdf_census.to_file('data/census_crop_yield.geojson', driver='GeoJSON')

C:\Users\sujit\AppData\Local\Temp\ipykernel_7516\2704136022.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  states_gdf['lon'] = states_gdf.geometry.centroid.x
C:\Users\sujit\AppData\Local\Temp\ipykernel_7516\2704136022.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  states_gdf['lat'] = states_gdf.geometry.centroid.y


In [22]:
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Crop Yield Choropleth</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body { margin: 0; padding: 20px; }
        #map { width: 800px; height: 600px; }
        #controls { margin-bottom: 20px; }
    </style>
</head>
<body>
    <div id="controls">
        <label>Crop:</label>
        <select id="cropSelect" onchange="updateMap()">
            <option value="CORN">Corn</option>
            <option value="SOYBEANS">Soybeans</option>
            <option value="WHEAT">Wheat</option>
            <option value="COTTON">Cotton</option>
        </select>
        <label>Year:</label>
        <select id="yearSelect" onchange="updateMap()">
            <option value="2020">2020</option>
            <option value="2021">2021</option>
            <option value="2022">2022</option>
            <option value="2023">2023</option>
            <option value="2024">2024</option>
        </select>
    </div>
    <div id="map"></div>

    <script>
        // Use relative path assuming served from the same directory structure
        const surveyDataUrl = './data/survey_crop_yield.geojson';
        let surveyData;

        // Load data once
        fetch(surveyDataUrl)
            .then(response => {
                if (!response.ok) throw new Error('Network response was not ok');
                return response.json();
            })
            .then(data => {
                surveyData = data;
                updateMap();  // Initial render
            })
            .catch(error => console.error('Error loading GeoJSON:', error));

        function updateMap() {
            const crop = document.getElementById('cropSelect').value;
            const year = document.getElementById('yearSelect').value;

            if (!surveyData) return;  // Wait for data to load

            // Filter data
            const filteredFeatures = surveyData.features.filter(f => 
                f.properties.COMMODITY_DESC === crop && f.properties.YEAR === parseInt(year)
            );
            const filteredData = {
                type: 'FeatureCollection',
                features: filteredFeatures
            };

            // Prepare data for Plotly
            const states = filteredData.features.map(f => f.properties.STATE_ALPHA);
            const values = filteredData.features.map(f => f.properties.VALUE || 0);

            const data = [{
                type: 'choropleth',
                locations: states,
                locationmode: 'USA-states',
                z: values,
                colorscale: 'YlGn',
                autocolorscale: false,
                marker: { line: { color: 'black', width: 1 } },
                colorbar: { title: 'Yield (Units Vary)' }
            }];

            const layout = {
                title: `${crop} Yield by State (${year})`,
                geo: {
                    scope: 'usa',
                    framecolor: 'black',
                    showframe: true
                },
                width: 800,
                height: 600
            };

            Plotly.newPlot('map', data, layout);
        }
    </script>
</body>
</html>
"""

# Write HTML file
with open('images/crop_yield_map.html', 'w') as f:
    f.write(html_content)

In [23]:
print("""
To view the interactive map locally:
1. Ensure 'data/survey_crop_yield.geojson' and 'images/crop_yield_map.html' are in the correct directory structure.
2. Run a local server from the project root directory:
   - Open a terminal in 'AgCommodityPricesEDA/'
   - Run: python -m http.server 8000
3. Open your browser and go to: http://localhost:8000/images/crop_yield_map.html
""")


To view the interactive map locally:
1. Ensure 'data/survey_crop_yield.geojson' and 'images/crop_yield_map.html' are in the correct directory structure.
2. Run a local server from the project root directory:
   - Open a terminal in 'AgCommodityPricesEDA/'
   - Run: python -m http.server 8000
3. Open your browser and go to: http://localhost:8000/images/crop_yield_map.html

